In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import rasterio
import numpy as np
import torch.nn as nn
import torch.optim as optim
from skimage.transform import resize
from tqdm import tqdm
import pandas as pd
import torchvision.transforms as transforms

In [5]:
def train_model(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, targets in tqdm(dataloader):
        inputs  = inputs.float().to(device)
        targets = targets.float().to(device)

        inputs[torch.isnan(inputs)] = 0
        targets[torch.isnan(targets)] = 0

        assert not torch.isnan(inputs).any(), "Input contains NaN values"
        assert not torch.isnan(targets).any(), "Target contains NaN values"
        
        optimizer.zero_grad()
        outputs_up = model(inputs)

        loss = criterion(outputs_up, targets).to(device)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

def validate_model(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for inputs, targets in tqdm(dataloader):
            inputs = inputs.float().to(device)
            targets =  targets.float().to(device)
            inputs[torch.isnan(inputs)] = 0
            targets[torch.isnan(targets)] = 0

            outputs = model(inputs)

            loss = criterion(outputs, targets)
            running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ChannelAttention(nn.Module):
    def __init__(self, channels, reduction=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        y = torch.cat([avg_out, max_out], dim=1)
        y = self.conv(y)
        return x * self.sigmoid(y)

class MSABlock(nn.Module):
    def __init__(self, channels):
        super(MSABlock, self).__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.ca = ChannelAttention(channels)
        self.sa = SpatialAttention()
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        residual = x
        out = self.relu(self.conv1(x))
        out = self.relu(self.conv2(out))
        out = self.conv3(out)
        out = self.ca(out)
        out = self.sa(out)
        out += residual
        return self.relu(out)

class MSAN(nn.Module):
    def __init__(self, in_channels=1, out_channels=1, num_features=64, num_blocks=8, upscale_factor=10):
        super(MSAN, self).__init__()
        self.conv_first = nn.Conv2d(in_channels, num_features, kernel_size=3, padding=1)
        
        self.body = nn.Sequential(
            *[MSABlock(num_features) for _ in range(num_blocks)]
        )
        
        self.conv_mid = nn.Conv2d(num_features, num_features, kernel_size=3, padding=1)
        
        # 修改上采样部分以实现10倍上采样
        self.upsampler = nn.Sequential(
            nn.Conv2d(num_features, num_features * 25, kernel_size=3, padding=1),
            nn.PixelShuffle(5),
            nn.Conv2d(num_features, num_features * 4, kernel_size=3, padding=1),
            nn.PixelShuffle(2),
            nn.Conv2d(num_features, out_channels, kernel_size=3, padding=1)
        )

    def forward(self, x):
        x = self.conv_first(x)
        residual = x
        x = self.body(x)
        x = self.conv_mid(x)
        x += residual
        x = self.upsampler(x)
        return x

Input shape: torch.Size([1, 1, 40, 52])
Output shape: torch.Size([1, 1, 400, 520])


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = MSAN(in_channels=1, out_channels=1, num_features=64, num_blocks=8, upscale_factor=10).to(device)

learning_rate = 0.0001
criterion = nn.L1Loss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-4)

num_epochs = 100
best_loss = float('inf')

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print(f'Model is training on: {next(model.parameters()).device}')
    train_loss = train_model(model, dataloader_train, criterion, optimizer, device)
    val_loss = validate_model(model, dataloader_val, criterion, device)
    print(f'Train Loss: {train_loss:.4f} | Validation Loss: {val_loss:.4f}')
    
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), 'path')
        print('Model saved!')


cuda
Epoch 1/100
Model is training on: cuda:0


100%|██████████| 115/115 [00:29<00:00,  3.87it/s]


Train Loss: 15.7937 | Validation Loss: 6.0207
Model saved!
Epoch 2/100
Model is training on: cuda:0


100%|██████████| 115/115 [00:29<00:00,  3.84it/s]


Train Loss: 4.2800 | Validation Loss: 2.8393
Model saved!
Epoch 3/100
Model is training on: cuda:0


...

Train Loss: 0.8468 | Validation Loss: 0.8150
Epoch 99/100
Model is training on: cuda:0


100%|██████████| 115/115 [00:29<00:00,  3.93it/s]


Train Loss: 0.8264 | Validation Loss: 0.7860
Epoch 100/100
Model is training on: cuda:0


100%|██████████| 115/115 [00:29<00:00,  3.85it/s]

Train Loss: 0.8320 | Validation Loss: 0.8540
